In [1]:
import cv2
import time
import torch
from playsound import playsound

from transformers import OwlViTProcessor, OwlViTForObjectDetection

# loading the downloaded model from pc
processor = OwlViTProcessor.from_pretrained("/home/ujwal/AI/owlvit-base-patch32")
model = OwlViTForObjectDetection.from_pretrained("/home/ujwal/AI/owlvit-base-patch32")

# Initialize the video capture device
cap = cv2.VideoCapture(0)

# Set the image size
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Set the interval between images (in seconds)
interval = 300  # 5 minutes

while True:
    # Get the current time
    current_time = time.time()

    # Capture a frame from the video device
    ret, frame = cap.read()

    # # Save the image

    image = frame
    texts = [["cat", "dog"]]
    inputs = processor(text=texts, images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Target image sizes (height, width) to rescale box predictions [batch_size, 2]
    target_sizes = torch.Tensor([image.shape[1::-1]])
    # Convert outputs (bounding boxes and class logits) to COCO API
    results = processor.post_process(outputs=outputs, target_sizes=target_sizes)

    # Track number of detected cats and dogs
    num_cats = 0
    num_dogs = 0

    for i in range(len(texts)):
        text = texts[i]
        boxes, scores, labels = results[i]["boxes"], results[i]["scores"], results[i]["labels"]

        # Print detected objects and rescaled box coordinates
        score_threshold = 0.1
        for box, score, label in zip(boxes, scores, labels):
            box = [round(i, 2) for i in box.tolist()]
            if score >= score_threshold:
                if text[label] == "cat":
                    num_cats += 1
                    print(f"Detected {text[label]} with confidence {round(score.item(), 3)} at location {box}")
                    if num_cats > 2:
                        print(f"ALERT: More than 2 cats detected!")
                        playsound("alarm.wav")
                elif text[label] == "dog":
                    num_dogs += 1
                    print(f"Detected {text[label]} with confidence {round(score.item(), 3)} at location {box}")
                    print("ALERT: Dog detected!")
                    playsound("alarm.wav")

    # Wait until the next capture time
    while time.time() - current_time < interval:
        time.sleep(1)

# Release the video capture device
cap.release()

/home/ujwal/anaconda3/envs/huggingface/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.
2023-02-21 12:30:15.706388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 12:30:15.865720: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRAR

KeyboardInterrupt: 